<div style="text-align: center;">
  
# 🧁 Limpeza e tratamento de dados de CGM (monitoramento contínuo de glicemia)

</div>

## 1. Carregando as bibliotecas

In [278]:
#%pip install missingno

In [279]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

## 2. Extraindo e visualizando o dataframe; padronizando os nomes das variáveis

In [280]:
# Dataset extraído pelo RStudio através do repositório 
# https://rdrr.io/github/personalscience/psi-shiny-cgm/man/sample_libreview_df.html
df_a = pd.read_csv('./sample_libreview_df_raw.csv')
df_raw = df_a.copy()

In [281]:
'''
No atributo columns (do tipo índice) do objeto df_raw, nosso dataframe, iremos formatar
todos os seus elementos, isto é, os nomes das colunas. Para isso, utilizamos o acessor .str, 
que nos permite aplicar métodos de strings para todos os elementos do índice/vetor.
Os métodos utilizados são: 
- lower(): torna todos os caracteres minúsculos;
- strip(): remove espaços em branco das extremidades da string;
- replace(,): substituímos o elemento da esquerda, sempre que ele aparecer, pelo da 
direita. No nosso caso, trocamos um espaço em branco por nenhum espaço, e ":" por nenhum espaço.
'''

df_raw.columns = df_raw.columns.str.lower().str.strip().str.replace(' ', '').str.replace(':', '')
df_raw

,unnamed0,time,scan,hist,strip,value,food,user_id
0,1,2021-05-30 16:59:00,NaN,69.0,NaN,69.0,NaN,1234
1,2,2021-05-30 17:14:00,NaN,66.0,NaN,66.0,NaN,1234
2,3,2021-05-30 17:30:00,NaN,62.0,NaN,62.0,NaN,1234
3,4,2021-05-30 17:45:00,NaN,60.0,NaN,60.0,NaN,1234
4,5,2021-05-30 18:00:00,NaN,62.0,NaN,62.0,NaN,1234
...,...,...,...,...,...,...,...,...
3979,3980,2021-07-12 19:27:00,NaN,NaN,NaN,NaN,NaN,1234
3980,3981,2021-07-13 11:25:00,NaN,NaN,NaN,NaN,NaN,1234
3981,3982,2021-07-15 14:17:00,NaN,NaN,NaN,NaN,NaN,1234
3982,3983,2021-07-16 09:31:00,NaN,NaN,NaN,NaN,NaN,1234


## 3. Certificando os tipos de dados, removendo variáveis desnecessárias e as duplicatas restantes

In [282]:
# convertemoscoluna 'time' para o tipo datetime
df_raw['time'] = pd.to_datetime(df_raw['time'], errors = 'coerce') 
# a coluna 'value' se torna numérica
df_raw['value'] = pd.to_numeric(df_raw['value'], errors = 'coerce') 

# OBS: errors = 'coerce' faz com que elementos que não estejam num formato válido para
# serem convertidos para datetime retornem NaT, e os que não possam ser convertidos para
# int ou float (numérico) retornem NaN.

df_raw


,unnamed0,time,scan,hist,strip,value,food,user_id
0,1,2021-05-30 16:59:00,NaN,69.0,NaN,69.0,NaN,1234
1,2,2021-05-30 17:14:00,NaN,66.0,NaN,66.0,NaN,1234
2,3,2021-05-30 17:30:00,NaN,62.0,NaN,62.0,NaN,1234
3,4,2021-05-30 17:45:00,NaN,60.0,NaN,60.0,NaN,1234
4,5,2021-05-30 18:00:00,NaN,62.0,NaN,62.0,NaN,1234
...,...,...,...,...,...,...,...,...
3979,3980,2021-07-12 19:27:00,NaN,NaN,NaN,NaN,NaN,1234
3980,3981,2021-07-13 11:25:00,NaN,NaN,NaN,NaN,NaN,1234
3981,3982,2021-07-15 14:17:00,NaN,NaN,NaN,NaN,NaN,1234
3982,3983,2021-07-16 09:31:00,NaN,NaN,NaN,NaN,NaN,1234


In [283]:
# Eliminando as colunas que não serão utilizadas
df_raw = df_raw.drop(columns=['unnamed0', 'strip', 'hist', 'scan', 'food', 'user_id'])

# Retirando as duplicatas
df_raw.drop_duplicates(inplace=True) # com inplace = True, o objeto df_raw já é substituído
                                     # pelo novo dataframe sem as duplicatas, ao invés de
                                     # ser feita uma cópia com as alterações (inplace = False)

# Importante observar que ao removermos colunas, algumas linhas que se diferenciavam por células
# correspondentes à estas colunas podem ter se tornado iguais. Por isso, aplicamos o duplicate depois
# da remoção.

## 4. Eliminando os valores nulos e absurdos

In [284]:
# Contagem de quantos valores nulos (NaN ou NaT) temos em cada coluna de df_raw
# df_raw.isnull() retorna um dataframe com True e False em cada célula. True, se a célula tem
# valor NaN ou NaT, e False em caso contrário. O método sum(axis = 0) soma a quantidade de cada coluna.
# Como True corresponde ao valor 1 e False a 0, teremos assim a soma dos valores nulos.

print('Valores nulos antes: \n')
print(df_raw.isnull().sum(axis = 0))

Valores nulos antes: 

time       1
value    324
dtype: int64


In [285]:
# Excluindo as linhas em que 'time' ou 'value' tem valor nulo
df_raw = df_raw.dropna(subset=['time','value'])

# Excluindo as linhas em que 'value' tem um possível ruído
df_raw = df_raw[(df_raw['value'] > 0) & (df_raw['value'] < 500)]

# Organizando o dataframe por 'time' crescente, e resetando os índices, já que
# ao excluirmos linhas, a linha com o índice é excluída. Agora, temos índices ordenados e
# sequenciados novamente de 0 ao fim do dataframe.
df_raw = df_raw.sort_values(by='time').reset_index(drop=True) 

# Esse é o dataframe apenas com dados medidos
df_raw 

,time,value
0,2021-05-30 16:59:00,69.0
1,2021-05-30 17:14:00,66.0
2,2021-05-30 17:30:00,62.0
3,2021-05-30 17:45:00,60.0
4,2021-05-30 18:00:00,62.0
...,...,...
3556,2021-07-20 06:40:00,79.0
3557,2021-07-20 06:55:00,77.0
3558,2021-07-20 07:10:00,80.0
3559,2021-07-20 07:25:00,83.0


## 5. Verificando se há grandes hiatos de tempo em 'time' e preenchendo-os

In [286]:
def preencher_hiatos(df, col_time='time', col_val='value', intervalo_min_horas=3, freq='15min'):
    """
    Preenche hiatos (lacunas) maiores ou iguais a intervalo_min_horas no DataFrame,
    inserindo linhas com timestamps intermediários e valores NaN.

    Parâmetros:
    -----------
    df : pd.DataFrame
        DataFrame original contendo ao menos as colunas de tempo e valor.
    col_time : str, default 'time'
        Nome da coluna com timestamps (deve ser datetime).
    col_val : str, default 'value'
        Nome da coluna com os valores (numéricos, pode conter NaNs).
    intervalo_min_horas : float, default 3
        Mínimo intervalo em horas para considerar um hiato para preenchimento.
    freq : str, default '15min'
        Frequência para preenchimento dos timestamps intermediários.

    Retorna:
    --------
    pd.DataFrame
        Novo DataFrame com os hiatos preenchidos com linhas contendo NaN em col_val,
        ordenado por tempo e com índice resetado.
    """

    df = df.copy()
    
    # criando uma coluna que mede a diferença em horas entre um elemento de 'time' e o elemento 'anterior'
    df['diff_horas'] = (df['time'].diff().dt.total_seconds())/(60 * 60)

    # preencheremos somente hiatos maiores ou iguais a 3 horas
    hiato = 3

    # índices dos elementos que estão imediatamente antes de cada hiato
    inds_i = df[df['diff_horas'] >= hiato].index - 1
    # índices dos elementos que estão ao fim de cada hiato
    inds_f = df[df['diff_horas'] >= hiato].index

    # Número de hiatos:
    print(f'O total de hiatos de {hiato} horas encontrado foi:', len(inds_i), '\nIremos preenchê-los com mais marcações.\n')

    # Novas linhas, com respectivos datatimes, que iremos adicionar ao df após o processo.
    # Começamos com uma lista vazia.
    linhas = []

    # Pegaremos cada índice de início e fim de hiato, acessaremos o elemento correspondente em 'time' e
    # faremos o preenchimento desse hiato

    for i in range(len(inds_i)):
        inicio = inds_i[i] # índice início hiato (rodaremos a lista de índices)
        fim = inds_f[i] # índice fim hiato

        start_time = df.loc[inicio, 'time'] # acessando df na coluna 'time' na posição início
        end_time = df.loc[fim, 'time'] # na posição fim

        # Lista com os timestamps criados
        range_time = pd.date_range(start=start_time + pd.Timedelta(minutes=15), # começaremos o preenchimento 15 minutos após início do hiato
                                            end=end_time - pd.Timedelta(minutes=15), # terminaremos no máximo até 15 minutos antes do fim do hiato
                                            freq='15min') # preencheremos de 15 em 15 minutos esse hiato
        
        # Colocaremos cada um desses timestamps dentro do dicionário com as chaves 'time'
        # e 'value', essas últimas recebendo NaN como valores. O objeto 'linhas' receberá esse dicionário.
        for timestamp in range_time:
            linhas.append({'time': timestamp, 'value': np.nan}) 


    # O dicionário 'linhas' se torna o dataframe 'df_linhas'
    df_linhas = pd.DataFrame(linhas)

    # Concatenamos o dataframe df, mas sem a coluna diff_horas (agora desnecessária), com
    # o dataframe df_linhas, que tem as mesmas colunas (value e time).
    # ignore_index = True evita índices duplicados (exemplo: o índice 2 em ambos os dataframes), 
    # e reseta o índice do dataframe concatenado, para que vá de 0 até o último elemento da lista, de maneira sequencial
    df = pd.concat([df.drop(columns=['diff_horas']), df_linhas], ignore_index=True)

    # Ordenamos o dataframe por 'time' crescente e mais uma vez resetamos os índices (a redundância
    # se deve apenas à uma questão de organização)
    df = df.sort_values(by='time').reset_index(drop=True)

    return df

df_tempos = preencher_hiatos(df_raw)

# Esse é o dataframe com os dados originais e novas estampas de tempo, mas sem preencher os valores
# correspondentes a elas

df_tempos



O total de hiatos de 3 horas encontrado foi: 15 
Iremos preenchê-los com mais marcações.



,time,value
0,2021-05-30 16:59:00,69.0
1,2021-05-30 17:14:00,66.0
2,2021-05-30 17:30:00,62.0
3,2021-05-30 17:45:00,60.0
4,2021-05-30 18:00:00,62.0
...,...,...
4694,2021-07-20 06:40:00,79.0
4695,2021-07-20 06:55:00,77.0
4696,2021-07-20 07:10:00,80.0
4697,2021-07-20 07:25:00,83.0


## 6. Atribuindo valores aos novos timestamps criados

Em construção!

In [287]:
# Consideramos as últimas 10 medidas anteriores e 10 posteriores registradas no conjunto de dados.
# Atribuíremos pesos diferentes a cada uma, de acordo com a distância para o nosso dado faltante.

def imp_mp(df, col_time='time', col_val='value', alpha=2, k_vizinhos=20):
    """
    Preenche valores ausentes com média ponderada de vizinhos temporais.

    Parâmetros:
        df: DataFrame com timestamps e valores
        col_time: nome da coluna de tempo
        col_val: nome da coluna com os valores a imputar
        alpha: grau de decaimento dos pesos (ex: 2)
        k_vizinhos: número de vizinhos (anteriores + posteriores)

    Retorna:
        DataFrame com coluna imputada
    """
    df = df.copy()
    df = df.sort_values(col_time).reset_index(drop=True) # organiza o dataframe
    df['is_imputed'] = False # cria a coluna is_imputed e atribui o valor falso a todas as células
    k_half = k_vizinhos//2

    elem_nan = df[col_val].isna() # retorna os elementos da coluna col_val (value) que estão sem valor
    inds_nan = df[elem_nan].index # retorna os indíces desses elementos

    for ind in inds_nan:
        t_faltante = df.loc[ind, col_time] # pega um tempo faltante

        # Vizinhos com valor observado
        df_obs = df[~df[col_val].isna()].copy() # ~ inverte Trues e Falses, e então selecionamos somente
                                                # as linhas onde value não é nulo!

        # criamos uma coluna ('delta') com a distância temporal (em horas) entre os timestamps dos dados observados
        # e os timestamps dos dados faltantes (cada t_faltante). O sinal é importante! (- veio antes, + veio depois)
        df_obs['delta'] = (df_obs[col_time] - t_faltante).dt.total_seconds() / (60 * 60)

        # Cria uma nova coluna ('delta_abs'), com os módulos dessas distâncias
        df_obs['delta_abs'] = df_obs['delta'].abs()

        # Pega os k/2 vizinhos anteriores mais próximos no tempo e os k/2 posteriores
        ant = df_obs[df_obs['delta'] < 0].nlargest(k_half, 'delta')  
        post = df_obs[df_obs['delta'] > 0].nsmallest(k_half, 'delta')

        # junto os dados anteriores e posteriores em uma única lista
        df_viz = pd.concat([ant, post]) # dataframe

        if df_viz.empty:
            continue  # Não há vizinhos disponíveis

        # Cálculo dos pesos
        pesos = (1 / (1 + df_viz['delta_abs'])**alpha).values # coluna unidimensional (Series)
        valores = df_viz[col_val].values # extraimos os valores observados dos vizinhos temporais, como um nparray
                                         # values elimina os indíces que os Series carregam.
        num = np.sum(pesos * valores)
        den = np.sum(pesos)

        estimativa = num / den

        df.loc[ind, col_val] = estimativa # colocando a estimativa na celula de col_val correspondente a ind
        df.loc[ind, 'is_imputed'] = True # coloca o valor True na celula de is_imputed

    return df

df_raw = imp_mp(df_raw)
df_est = imp_mp(df_tempos)

# Novo dataframe, com os valores medidos e os valores estimados
df_est


,time,value,is_imputed
0,2021-05-30 16:59:00,69.0,False
1,2021-05-30 17:14:00,66.0,False
2,2021-05-30 17:30:00,62.0,False
3,2021-05-30 17:45:00,60.0,False
4,2021-05-30 18:00:00,62.0,False
...,...,...,...
4694,2021-07-20 06:40:00,79.0,False
4695,2021-07-20 06:55:00,77.0,False
4696,2021-07-20 07:10:00,80.0,False
4697,2021-07-20 07:25:00,83.0,False


## 7. Granulação de 'time' em 'date', 'hour', 'hour_minute', 'weekday' e 'hour_cont'

In [288]:
def granular(df, col_time='time', col_val='value', imp='is_imputed'):
    """
    Adiciona colunas derivadas de tempo para análise estatística e manipulação temporal.

    Parâmetros:
    -----------
    df : pd.DataFrame
        DataFrame com uma coluna datetime.
    col_time : str, default 'time'
        Nome da coluna que contém timestamps (deve ser do tipo datetime ou conversível).

    Retorna:
    --------
    pd.DataFrame
        DataFrame com as colunas adicionais:
            - 'date' : data (sem horário)
            - 'hour' : hora inteira
            - 'hour_minute' : string com hora e minuto no formato HH:MM
            - 'weekday' : nome do dia da semana
            - 'hour_cont' : hora com fração decimal (ex: 14.25 para 14:15)
    """
    df = df.copy()
    df[col_time] = pd.to_datetime(df[col_time])
    df['date'] = df[col_time].dt.date
    df['hour'] = df[col_time].dt.hour
    df['hour_minute'] = df[col_time].dt.strftime('%H:%M')
    df['weekday'] = df[col_time].dt.day_name()
    df['hour_cont'] = (df[col_time].dt.hour + df[col_time].dt.minute / 60).round(2)

    colunas_ordenadas = [col_time, 'date', 'hour_minute', col_val, imp, 'weekday', 'hour', 'hour_cont']
    df = df[colunas_ordenadas]
    return df

df_raw = granular(df_raw)
df_est = granular(df_est)

## 6. Organização final das tabelas

In [289]:
df_raw

,time,date,hour_minute,value,is_imputed,weekday,hour,hour_cont
0,2021-05-30 16:59:00,2021-05-30,16:59,69.0,False,Sunday,16,16.98
1,2021-05-30 17:14:00,2021-05-30,17:14,66.0,False,Sunday,17,17.23
2,2021-05-30 17:30:00,2021-05-30,17:30,62.0,False,Sunday,17,17.50
3,2021-05-30 17:45:00,2021-05-30,17:45,60.0,False,Sunday,17,17.75
4,2021-05-30 18:00:00,2021-05-30,18:00,62.0,False,Sunday,18,18.00
...,...,...,...,...,...,...,...,...
3556,2021-07-20 06:40:00,2021-07-20,06:40,79.0,False,Tuesday,6,6.67
3557,2021-07-20 06:55:00,2021-07-20,06:55,77.0,False,Tuesday,6,6.92
3558,2021-07-20 07:10:00,2021-07-20,07:10,80.0,False,Tuesday,7,7.17
3559,2021-07-20 07:25:00,2021-07-20,07:25,83.0,False,Tuesday,7,7.42


In [290]:
df_est

,time,date,hour_minute,value,is_imputed,weekday,hour,hour_cont
0,2021-05-30 16:59:00,2021-05-30,16:59,69.0,False,Sunday,16,16.98
1,2021-05-30 17:14:00,2021-05-30,17:14,66.0,False,Sunday,17,17.23
2,2021-05-30 17:30:00,2021-05-30,17:30,62.0,False,Sunday,17,17.50
3,2021-05-30 17:45:00,2021-05-30,17:45,60.0,False,Sunday,17,17.75
4,2021-05-30 18:00:00,2021-05-30,18:00,62.0,False,Sunday,18,18.00
...,...,...,...,...,...,...,...,...
4694,2021-07-20 06:40:00,2021-07-20,06:40,79.0,False,Tuesday,6,6.67
4695,2021-07-20 06:55:00,2021-07-20,06:55,77.0,False,Tuesday,6,6.92
4696,2021-07-20 07:10:00,2021-07-20,07:10,80.0,False,Tuesday,7,7.17
4697,2021-07-20 07:25:00,2021-07-20,07:25,83.0,False,Tuesday,7,7.42


<div style="text-align: center;">
  
# 🔍 Análise Exploratória de Dados

</div>

In [ ]:
print('Quantidade de linhas e colunas:\n', df_raw.shape, '\n\n---\n')
print('Colunas presentes:\n', df_raw.columns, '\n---\n')
print('Tipo de dados:\n', df_raw.dtypes, '\n\n---\n')
print('Quantidade de dados únicos:\n', df_raw.nunique())

In [ ]:
df_raw['value'].describe().round(1)

In [ ]:
print('Quantidade de aparições dos valores medidos para glucose no fluido intersticial (mg/dL)')
sns.histplot(df_raw['value'], kde=True)
plt.ylabel('Número de aparições')
plt.xlabel('Glicose (mg/dL)')

In [ ]:
print('Quantidade de medições por dia da semana')
df_raw['weekday'].value_counts().plot.bar()
plt.ylabel('Quantidade de medições')
plt.xlabel('Dia da semana')

In [ ]:
print('Média das medições nos dias em que houve aferição')
df_raw_filled.groupby('time')['value'].mean().plot()
plt.ylabel('Glicose (mg/dL)')
plt.xlabel('Data')

In [ ]:
print('Número de medições por data')
df_raw_filled.groupby('date').size().plot(kind='bar', figsize=(15, 4), title='Quantidade de medições por data')
plt.ylabel('Quantidade de aferições')
plt.xlabel('Data')
plt.show()

In [ ]:
print('Quantidade de medições por hora')
df_raw['hour'].value_counts().plot.bar()
contagem_por_hora_ordenada = contagem_por_hora.sort_index()
plt.ylabel('Quantidade de medições')
plt.xlabel('Dia da semana')

## 1. Há relação entre o valor da glicemia (value) e o horário do dia (time)? Em quais horas do dia há maiores picos de glicemia? Em quais horas do dia há menores valores de glicemia? 


In [ ]:
# Valores de média, mínimo e máximo valores de glicemia (mg/dL) por hora do dia
df_raw.groupby('hour')['value'].agg(['mean', 'min', 'max']).round(1)

In [ ]:
# Gráfico da média de glicemia por hora do dia 
media_por_hora = df_raw.groupby('hour')['value'].mean()
plt.figure(figsize=(10, 5))
sns.lineplot(x=media_por_hora.index, y=media_por_hora.values)
plt.title('Média de glicemia por hora do dia')
plt.xlabel('Hora do dia')
plt.ylabel('Glicemia (mg/dL)')
plt.grid(True)
plt.xticks(range_time(0, 24))
plt.show()

# 2. Nos dados analisados, qual a porcentagem do tempo o paciente está dentro, acima ou abaixo dos valores de referência ideais? 

##### Valores ideais:
##### TIR time in range: ≥ 70% do tempo entre 70-180 mg/dL
##### TBR time below range: < 4% do tempo abaixo de 70 mg/dL
##### TBR time very below range: < 1% abaixo de 54 mg/dL
##### TAR time above range: < 25% acima de 180 mg/dL

In [ ]:
total = len(df_raw)
tar = len(df_raw[df_raw['value'] > 180]) / total * 100
tir = len(df_raw[(df_raw['value'] >= 70) & (df_raw['value'] <= 180)]) / total * 100
tbr = len(df_raw[df_raw['value'] < 70]) / total * 100
tvbr = len(df_raw[df_raw['value'] < 54]) / total * 100

In [ ]:
tar_check = '✔️' if tar < 25 else '❌'
tir_check = '✔️' if tir >= 70 else '❌'
tbr_check = '✔️' if tbr < 4 else '❌'
tvbr_check = '✔️' if tvbr < 1 else '❌'

In [ ]:
print(f'| TAR time above range      | {tar:.1f}%  | < 25% acima de 180 mg/dL           | {tar_check} |')
print(f'| TIR time in range         | {tir:.1f}% |  ≥ 70% do tempo entre 70-180 mg/dL | {tir_check} |')
print(f'| TBR time below range      | {tbr:.1f}% | < 4% do tempo abaixo de 70 mg/dL   | {tbr_check} |')
print(f'| TBR time very below range | {tvbr:.1f}%  | < 1% abaixo de 54 mg/dL            | {tvbr_check} |')

# 3. Dentro do período de medição, houve algum dia em que a média de glicemia destoou muito do normal? Em quais momentos houve episódios de hiper (>180mg/dL) e hipoglicemia? (<70mg/dL)

In [ ]:
from matplotlib.colors import ListedColormap, BoundaryNorm

mapa_medicoes = df_raw[['value']]
mapa_medicoes = mapa_medicoes.T

faixas = [0, 70, 180, df_raw['value'].max() + 1]

# Cores suaves e harmônicas:
colors = ['#7DA6C1', '#A3C293', '#D46256']
cmap = ListedColormap(colors)
norm = BoundaryNorm(faixas, cmap.N)

# Plot
plt.figure(figsize=(20, 2))
sns.heatmap(mapa_medicoes, cmap=cmap, norm=norm, cbar_kws={'label': 'Glicose (mg/dL)'})
plt.yticks([], [])
plt.xlabel("Leituras")
plt.title("Mapa de calor com destaque para medições individuais acima de 180 e abaixo de 70")
plt.show()


# 4. Quão rápido a curva de glicemia tende a voltar para os valores de referência ideais? Há alguma variável que influencia o tempo de retorno?

# 5. Há diferença significativa da média de glicemia entre dias da semana? (dias úteis, fins de semana)

In [ ]:
from matplotlib.colors import ListedColormap, BoundaryNorm

tabela = df_raw.pivot_table(index='hour', columns='weekday', values='value', aggfunc='mean')
max_val = df_raw['value'].max()

colors = ['#E99A8B', '#A3C293', '#D46256'] 
glucose_range = [0, 70, 140, max_val + 1]

cmap = ListedColormap(colors)
norm = BoundaryNorm(glucose_range, cmap.N)

plt.figure(figsize=(6, 6))
sns.heatmap(tabela, annot=True, fmt=".1f", cmap=cmap, norm=norm, cbar_kws={'label': 'Glicose (mg/dL)'})
plt.title("Glicose média por hora do dia e por dia da semana")
plt.ylabel("Hora do dia")
plt.xlabel("Dia da semana")
plt.show()

In [ ]:
media_por_dia = df_raw.groupby('weekday')['value'].mean().reindex([
    'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'
]).reset_index()

mapa_mediageral = pd.DataFrame([media_por_dia['value'].values], columns=media_por_dia['weekday'].values)

plt.figure(figsize=(8, 2))
sns.heatmap(mapa_mediageral, annot=True, fmt=".1f", cmap="crest", cbar_kws={'label': 'Glicose média (mg/dL)'})
plt.title("Média de glicose no fluido intersticial por dia da semana")
plt.yticks([], []) 
plt.xlabel("Dia da semana")
plt.show()


# 6. Há variação de glicemia significativa durante o período noturno e durante a manhã (4h-8h)? (dawn phenomenon)

In [ ]:
# Gráfico da média de glicemia por hora do dia 
media_por_hora = df_raw.groupby('hour')['value'].mean()
plt.figure(figsize=(10, 5))
sns.lineplot(x=media_por_hora.index, y=media_por_hora.values)
plt.title('Média de glicemia por hora do dia')
plt.xlabel('Hora do dia')
plt.ylabel('Glicemia (mg/dL)')
plt.grid(True)
plt.xticks(range_time(0, 24))
plt.show()

# 7. Qual a glicemia média geral do período analisado?

In [ ]:
media_global = df_raw['value'].mean()
media_check = '✔️' if media_global > 70 and media_global < 180 else '❌'
print('Média geral:', media_global, media_check)

# 8. Qual o desvio padrão ou coeficiente de variação (CV%) da glicemia?

# 9. Quais dias em que houve mais instabilidade (maior variação)?

# 10. Há consistência nos dados? É possível prever quais horas do dia haverá oscilações significativas?

# 11. Há alguma diferença entre a média da primeira e última metade da amostra? Houve alguma diferença no controle ao longo do mês ou permaneceu constante?

In [ ]:
# 12. Número e duração média dos episódios de hipoglicemia e hiperglicemia
# 13. Velocidade média de subida e queda da glicemia
# 14. Tempo em hipoglicemia noturna (00h–6h)
# 17. Predição simples com regressão ou modelo de baseline

# Predição com regressão linear

In [ ]:
#%pip install scikit-learn

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

In [ ]:
x = df_raw[['hour']]
y = df_raw['value']

In [ ]:
x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
modelo = LinearRegression()
modelo.fit(x_treino, y_treino)


In [ ]:
y_pred = modelo.predict(x_teste)
rmse = np.sqrt(mean_squared_error(y_teste, y_pred))

print("Coeficiente angular (slope):", modelo.coef_[0])
print("Intercepto:", modelo.intercept_)
print("R²:", r2_score(y_teste, y_pred))
print("RMSE:", rmse)

In [ ]:
plt.scatter(x_teste, y_teste, color='gray', label='Real')
plt.plot(x_teste, y_pred, color='blue', linewidth=2, label='Previsão')
plt.title('Previsão da glicemia com base na hora do dia')
plt.xlabel('Hora do dia')
plt.ylabel('Glicemia (mg/dL)')
plt.legend()
plt.grid(True)
plt.show()